## Adding SNOTEL data to meta_data

- Currently working on calling API with lat/long. Haven't tested API call yet

In [ ]:
!gsutil cp gs://flow_datasets/01010500.json .

Copying gs://flow_datasets/01010500.json...
/ [1 files][ 14.4 KiB/ 14.4 KiB]                                                
Operation completed over 1 objects/14.4 KiB.                                     


In [ ]:
import json
import requests

def add_snotel_station_meta(meta_data_path, threshold = 200):
  """
  meta_data_path:
  threshold: is max distance to associate SNOTEL site in kilometers
  """
  with open(meta_data_path) as f:
    meta_data_file = json.load(f)
  lat = meta_data_file["latitude"]
  long1 = meta_data_file["logitude"]
  base_url = "https://powderlines.kellysoftware.org/api/closest_stations?lat={}&lng={}"
  response = requests.get(base_url.format(lat, long1)) # https://powderlines.kellysoftware.org/api/closest_stations?lat=39.7392&lng=-104.9903&count=3&data=true&days=3
  sno_response_json = json.loads(response.text)
  lat_close_station = sno_response_json[0]["station_information"]["location"]["lat"]
  print(sno_response_json)
  long_close_station = sno_response_json[0]["station_information"]["location"]["lng"]
  dis = haversine(lat, long1, lat_close_station, long_close_station)
  print(dis)
  if dis > 330:
    print("No close SNOTEL site distance is: " + str(dis))
  else:
    meta_data_file["snotel"] = sno_response_json[0]["station_information"]
  return meta_data_file

In [ ]:
from math import radians, sin, cos, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
  """
  Calculate the great circle distance between two points
  on the earth (specified in decimal degrees). In kilometers
  """
  # convert decimal degrees to radians
  lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

  # haversine formula
  dlon = lon2 - lon1
  dlat = lat2 - lat1
  a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
  c = 2 * asin(sqrt(a))
  r = 6371 # Radius of earth in kilometers. Use 3956 for miles
  return c * r

In [ ]:
add_snotel_station_meta("/content/01010500.json")

[{'station_information': {'name': 'North Rapid Creek', 'triplet': '920:SD:SNTL', 'elevation': 6130, 'location': {'lat': 44.20617, 'lng': -103.78758}}}, {'station_information': {'name': 'Blind Park', 'triplet': '354:SD:SNTL', 'elevation': 6890, 'location': {'lat': 44.10772, 'lng': -103.97688}}}, {'station_information': {'name': 'Cole Canyon', 'triplet': '982:WY:SNTL', 'elevation': 5910, 'location': {'lat': 44.48632, 'lng': -104.41057}}}]
3857.1859259428584
No close SNOTEL site distance is: 3857.1859259428584


{'id': '01010500',
 'latitude': 47.11305556,
 'logitude': -69.0880556,
 'name': 'St. John River at Dickey, Maine',
 'state': 'ME',
 'timeZoneInfo': "{'zoneOffset': '-05:00', 'zoneAbbreviation': 'EST'}",
 'elevation': {'results': [{'elevation': 181.9550170898438,
    'location': {'lat': 47.11305556, 'lng': -69.0880556},
    'resolution': 19.08790397644043}],
  'status': 'OK'},
 'soil_data': {'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-69.0880556, 47.11305556]},
  'properties': {'layers': [{'name': 'bdod',
     'unit_measure': {'d_factor': 100,
      'mapped_units': 'cg/cm³',
      'target_units': 'kg/dm³',
      'uncertainty_unit': ''},
     'depths': [{'range': {'top_depth': 0,
        'bottom_depth': 5,
        'unit_depth': 'cm'},
       'label': '0-5cm',
       'values': {'Q0.05': None,
        'Q0.5': None,
        'Q0.95': None,
        'mean': None,
        'uncertainty': None}},
      {'range': {'top_depth': 5, 'bottom_depth': 15, 'unit_depth': 'cm'},
   

## Getting SNOTEL on daily basis

In [ ]:
import pandas as pd
def get_snotel_daily_date(station_id, start_date, end_date):
  base_url = "https://powderlines.kellysoftware.org/api/station/{}?start_date={}&end_date={}"
  response = requests.get(base_url.format(station_id, start_date, end_date))
  json_res = json.loads(response.text)
  return pd.DataFrame(json_res["data"])

get_snotel_daily_date("335:CO:SNTL", "2019-02-01", "2020-02-01")


,Date,Snow Water Equivalent (in),Change In Snow Water Equivalent (in),Snow Depth (in),Change In Snow Depth (in),Observed Air Temperature (degrees farenheit)
0,2019-02-01,11.5,0.0,49,-1,15.8
1,2019-02-02,11.5,0.0,49,0,16.3
2,2019-02-03,11.5,0.0,47,-2,21.0
3,2019-02-04,11.5,0.0,46,-1,21.0
4,2019-02-05,11.5,0.0,45,-1,24.4
...,...,...,...,...,...,...
361,2020-01-28,12.3,0.2,52,0,10.0
362,2020-01-29,12.3,0.0,51,-1,10.4
363,2020-01-30,12.4,0.1,51,0,9.5
364,2020-01-31,12.7,0.3,53,2,6.4
